# Synthetic Data Generation with GAN

This notebook implements a Generative Adversarial Network (GAN) to generate synthetic application usage data.

## Project Structure
- The main functions are in `data_gen_utils.py`  
- The original data is located in `./data/screentime_analysis.csv`

In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Import our custom functions
from data_gen_utils import (
    build_generator, build_discriminator, build_gan,
    train_gan, preprocess_data, generate_synthetic_data
)


## 1. Data Loading and Preparation

We load the application usage data and prepare it for GAN training.

In [5]:
# Load data
data = pd.read_csv('../data/screentime_analysis.csv')
print("Raw data preview:")
print(data.head())

# Data preprocessing
normalized_data, scaler = preprocess_data(data)
print("\nShape of normalized data:", normalized_data.shape)

Raw data preview:
         Date        App  Usage (minutes)  Notifications  Times Opened
0  2024-08-07  Instagram               81             24            57
1  2024-08-08  Instagram               90             30            53
2  2024-08-26  Instagram              112             33            17
3  2024-08-22  Instagram               82             11            38
4  2024-08-12  Instagram               59             47            16

Shape of normalized data: (200, 3)


## 2. GAN Configuration

Setting up and building the Generator and Discriminator models.

![GAN Discriminator](../image/GAN_discriminator.png)

In [6]:
# Model parameters
latent_dim = 100  # dimension of the latent space

# Build models
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Display architectures
print("Generator architecture:")
generator.summary()
print("\nDiscriminator architecture:")
discriminator.summary()


/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/synthetic-data-3D4-9DEH-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/synthetic-data-3D4-9DEH-py3.12/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
2025-07-29 11:57:37.178636: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Generator architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 182,659 (713.51 KB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 512)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 166,401 (650.00 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 166,401 (650.00 KB)

## 3. Model Training

Training the GAN on our normalized data.

In [7]:
# Train the GAN
train_gan(
    gan=gan,
    generator=generator,
    discriminator=discriminator,
    data=normalized_data,
    nb_epochs=100,  # Adjust as needed
    batch_size=128,
    latent_dim=latent_dim
)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/synthetic-data-3D4-9DEH-py3.12/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: D Loss: [0.69074875 0.75      ], G Loss: 0.6626264452934265
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━

## 4. Synthetic Data Generation

Using the trained model to generate new data.

In [8]:
# Generate new data
feature_names = [col for col in data.columns if col not in ['Date', 'App']]
generated_df = generate_synthetic_data(
    generator=generator,
    scaler=scaler,
    n_samples=1000,
    latent_dim=latent_dim,
    feature_names=feature_names
)

print("Preview of generated data:")
print(generated_df.head())

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Preview of generated data:
   Usage (minutes)  Notifications  Times Opened
0        56.830372     129.494659     26.542692
1         1.001729       0.000054      1.000018
2       114.038704     144.770432     98.779320
3       115.785591     144.567856     98.514954
4       118.233452     146.870361     98.987000
